In [ ]:
pip install nomic

In [1]:
from nomic import atlas
import numpy as np
import json
from data.models import ProductSection
import os
from tqdm import tqdm
import nomic
nomic.login('cISRutntzMcGixIPpXDCBYrstcwpUbwBs0AX43BY4mUHt')

In [2]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
TGA_labels = ProductSection.objects.filter(label_product__drug_label__source="TGA").filter(bert_vector__isnull=False)
embeddings = []
metadata = []
cap = 48000
for i, ps in enumerate(tqdm(TGA_labels)):
    # cast to Numpy array
    if i >= cap:
        break
    embeddings.append(np.array(json.loads(ps.bert_vector)))
    doc = dict(ps.as_search_document())
    doc.pop("text_embedding")
    metadata.append(doc)
arr = np.array(embeddings)

 57%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                                                      | 48000/83725 [54:24<40:29, 14.70it/s]


In [4]:
project = atlas.map_embeddings(
    embeddings=arr,
    data=metadata,
    id_field="id",
    colorable_fields=["drug_label_source", "section_name"],
    topic_label_field="section_text",
    name="SearchRx",
    description="Embeddings of drug label sections",
    reset_project_if_exists=True
)

2023-04-18 16:34:10.708 | INFO     | nomic.project:__init__:871 - Found existing project `SearchRx` in organization `cole_crawford`. Clearing it of data by request.
2023-04-18 16:34:12.547 | INFO     | nomic.project:_create_project:965 - Creating project `SearchRx` in organization `cole_crawford`
2023-04-18 16:34:13.780 | INFO     | nomic.atlas:map_embeddings:100 - Uploading embeddings to Atlas.
48it [00:36,  1.33it/s]                                                                                                                                                                                                                                                                     
2023-04-18 16:34:54.369 | INFO     | nomic.project:_add_data:1577 - Upload succeeded.
2023-04-18 16:34:54.387 | INFO     | nomic.atlas:map_embeddings:119 - Embedding upload succeeded.
2023-04-18 16:34:55.477 | INFO     | nomic.project:create_index:1282 - Created map `SearchRx` in project `SearchRx`: https://atlas.n